In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
BASE_URL = "https://fastwork.co"

In [3]:
main_cat_urls = [
    'https://fastwork.co/design-graphic',
    'https://fastwork.co/marketing-advertising',
    'https://fastwork.co/writing-translation',
    'https://fastwork.co/photography-video',
    'https://fastwork.co/web-programming',
    'https://fastwork.co/consultant',
    'https://fastwork.co/operation'
]

In [4]:
def retrieve_products(subcat_slug):
    url = "https://f24luz3xrm-dsn.algolia.net/1/indexes/th_prod_products/query"
    params = {
        "x-algolia-agent":"Algolia for JavaScript (3.35.1); Browser",
        "x-algolia-application-id":"F24LUZ3XRM",
        "x-algolia-api-key":"a639c2213baa3a66e57cee665d7e17a6"
    }
    data = {
        "attributesToRetrieve": ["*"],
        "facets": ["subcategory.title","subcategory.slug","base_price","user.first_name","tags"],
        "filters":"(subcategory.slug: {})".format(subcat_slug),
        "hitsPerPage":"100",
        "page":"0",
        "query":"",
    }
    response = requests.post(url,params=params,json=data)
    return response

In [ ]:
df_lists = []
for main_cat_url in main_cat_urls:
    response = requests.get(main_cat_url)
    soup = BeautifulSoup(response.content, "html.parser")
    dom = soup.find_all('div',attrs={"class":"category-list"})
    sub_cat_links = dom[0].find_all('a')
    print(main_cat_url)
    
    for link in sub_cat_links:
        sub_cat_slug = link.attrs['href'].replace("/","")
        print("##{}".format(sub_cat_slug))
        
        response = retrieve_products(sub_cat_slug)
        data = response.json()['hits']
        
        df = pd.DataFrame(data)
        df['category'] = main_cat_url
        df['subcategory']= sub_cat_slug
        df_lists.append(df)
        
df_all = pd.concat(df_lists)

https://fastwork.co/design-graphic
##logo-design
##illustration
##social-media-banner
##character-design
##packaging
##fanart
##illustration-portrait
##engineering-structural-design
##caricature
##architect-and-interior
##cartoon-drawing
##presentation
##vector-illustration
##publication
##vector
##storyboard
##fashion-design
##tattoo-design
##infographics
##map-design
##photoshop
##game-character-bg
##sticker
##perspective
##3d-model
##line-sticker
##portfolio-resume
##corporate-identity
##landscape
##nft-art
##design-graphic-other
https://fastwork.co/marketing-advertising
##seo
##promote-page
##reviews
##blogger-netidol
##data-analyst
##questionnaire
##social-media-ads
##google-ads
##content-marketing


In [ ]:
print(df_all)

In [ ]:
plt.figure(figsize=(16, 9))
df_all.plot.scatter(x="subcategory",y="base_price")